In [8]:
import thunder
import torch

In [11]:
from thunder.benchmarks import NanoGPTBlockBenchmark, NanoGPTConfig
from thunder.executors.nvfuserex import nvfuserex

config = NanoGPTConfig(seq_len=2048, n_head=96, n_embd=12288)
bench = NanoGPTBlockBenchmark(
  batchdims=(1,), config=config, device="cuda:0", dtype=thunder.bfloat16, requires_grad=True
)
args, kwargs = bench.make_batch()

jfn = thunder.jit(
    bench.fn(),
    executors=[nvfuserex],
    nv_enable_sdpa=True,
    nv_enable_matmul=True,
    nv_enable_linear=True,
    disable_replace_uniform=True,
)
out = jfn(*args, **kwargs)
grads = torch.randn_like(out, device="cuda", dtype=torch.bfloat16)
out.backward(grads)
print(thunder.last_traces(jfn)[-1].python_ctx()['nvFusion0'].last_used)
print(thunder.last_backward_traces(jfn)[-1].python_ctx()['nvFusion0'].last_used)


def nvfuser_fusion_id2(fd : FusionDefinition) -> None :
    T0 = fd.define_tensor(shape=[1, 2048, 12288], contiguity=[None, True, True], dtype=DataType.BFloat16, is_cpu=False, stride_order=[2, 1, 0])
    T1 = fd.define_tensor(shape=[12288], contiguity=[True], dtype=DataType.BFloat16, is_cpu=False, stride_order=[0])
    T2 = fd.define_tensor(shape=[12288], contiguity=[True], dtype=DataType.BFloat16, is_cpu=False, stride_order=[0])
    T3 = fd.define_tensor(shape=[36864, 12288], contiguity=[True, True], dtype=DataType.BFloat16, is_cpu=False, stride_order=[1, 0])
    T4 = fd.define_tensor(shape=[36864], contiguity=[True], dtype=DataType.BFloat16, is_cpu=False, stride_order=[0])
    T5 = fd.define_tensor(shape=[12288, 12288], contiguity=[True, True], dtype=DataType.BFloat16, is_cpu=False, stride_order=[1, 0])
    T6 = fd.define_tensor(shape=[12288], contiguity=[True], dtype=DataType.BFloat16, is_cpu=False, stride_order=[0])
    T7 = fd.define_tensor(shape=[12288], contiguity=[True], dtype